(1) Initialization.

In [1]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

logging.basicConfig(filename="log.txt", level=logging.DEBUG);

(2) Generate parameters for DH algorithm. Create ControlER public key. Create ControlER Private key

In [2]:
# Generate parameters for dh
parameters = dh.generate_parameters(generator=2, key_size=2024, backend=default_backend()); #return:DHParameters
logging.debug(" ControlER: Parameters generated.");

# Creating the private and public keys
cont_private_key = parameters.generate_private_key();
logging.debug(" ControlER: Created the private key.");

cont_public_key = cont_private_key.public_key();
logging.debug(" ControlER: Created the private key.");

(3) Write ControlER public key in bytes and send in the communication channel.

In [3]:
cont_public_key_bytes = cont_public_key.public_bytes(serialization.Encoding.PEM, serialization.PublicFormat.SubjectPublicKeyInfo);
with open('channel', 'wb') as f:
    f.write(cont_public_key_bytes)
    f.close()
logging.debug(" ControlER: Sent the public key public key to the communication server.");

(4) ---

(5) ---

(6) ---

(7) ---

(8) ---

(9) ---

(10) Receive Delentture public key bytes from the communication channel. Create the Delentture public key from loaded bytes.

In [4]:
with open('channel', 'rb') as f:
    dele_public_key_bytes = f.read();
logging.debug(" ControlER: Loaded the Delentture public key bytes from the channel.");

dele_public_key = load_pem_public_key(dele_public_key_bytes, default_backend())
logging.debug(" Delentture: Created the ControlER public key from the loaded bytes.");

(11) Generate the shared key between the two parties. In this end - a mix of Delentture public key and ControlER private key

In [5]:
shared_key = cont_private_key.exchange(dele_public_key);
logging.debug(" ControlER: Generated the shared key:");
logging.info(shared_key);

(12) Hashing - generate secret cipher key (from the shared key between two parties) and secret nonce

In [6]:
secret_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32, # 32 bytes is the highest level of encryption for AES-GCM
    salt=None,
    info=b'secret', # handshake data
).derive(shared_key)

logging.debug(" ControlER: Generated the secret key:");
logging.info(secret_key);

nonce = HKDF(
    algorithm=hashes.SHA256(),
    length=12, # must use 12 bits, because the AES-GCM operates on nonces with 12 bytes.
    salt=None,
    info=b'nonce', # handshake data
).derive(secret_key);

logging.debug(" ControlER: Generated the secret nonce:");
logging.info(nonce);

(13) Load original dataset into bytes.

In [7]:
with open('dataset.csv', 'rb') as file:
    original_dataset = file.read()
    
logging.debug(" ControlER: Loaded the original dataset into bytes.");

(14) Encrypt original dataset and generate corresponding ciphertext.

In [8]:
aesgcm = AESGCM(secret_key);

original_dataset_ciphertext = aesgcm.encrypt(nonce, original_dataset, secret_key);

logging.debug(" ControlER: Encrypted original dataset to ciphertext.");

(15) Send the ciphertext to the communication channel.

In [9]:
with open('channel', 'wb') as f:
    f.write(original_dataset_ciphertext);
    f.close()
logging.debug(" ControlER: Sent the original dataset ciphertext to the communication server.");

(16) ---

(17) ---

(18) ---

(19) Do the same for the original description.

In [10]:
with open('description.csv', 'rb') as file:
    original_description = file.read()
    
logging.debug(" ControlER: Loaded the original description into bytes.");

original_description_ciphertext = aesgcm.encrypt(nonce, original_description, secret_key);

logging.debug(" ControlER: Encrypted original description to ciphertext.");

with open('channel', 'wb') as f:
    f.write(original_description_ciphertext);
    f.close()
logging.debug(" ControlER: Sent the original description ciphertext to the communication server.");

(20) ---

(21) ---

(22) ---

(23) Receive, decrypt and save the first data analysis pdf.

In [11]:
with open('dataset.csv', 'rb') as file:
    original_dataset = file.read()
    
logging.debug(" ControlER: Loaded the original dataset into bytes.");
    
logging.debug(" ControlER: Saved the plot1 pdf successfuly.");

(24) ---

(25) Receive, decrypt and save the second data analysis pdf.

In [12]:
with open('channel', 'rb') as f:
    plot2_ciphertext = f.read();
    
logging.debug(" ControlER: Received the encrypted plot2 pdf successfuly.");

decrypted_plot2_bytes = aesgcm.decrypt(nonce, plot2_ciphertext, secret_key);
logging.debug(" ControlER: Decrypted the plot2 pdf successfuly.");

with open('decrypted_plot2.pdf', 'wb') as f:
    f.write(decrypted_plot2_bytes)
    
logging.debug(" ControlER: Saved the plot2 pdf successfuly.");